# Learning About Probabilities and Related ML Methods

In this blog we will be discussing Naive Bayes and Logistic Regression.
The reason for such a selection of topics is the comonality that both of these algorithms share.
Both of these algorithms are probabilistic classifiers.
This means that both of these methods can output not only their prediction for an outcome, but also the probability of the outcome.
We will see the math behind how these models go about doing it as we progress through the blog.
First, we should discuss the data.

## Diabetes Dataset
The current version of the diabetes dataset was taken from [kaggle](https://www.kaggle.com/datasets/mathchi/diabetes-data-set/data).
However, it is only a subset of the full dataset owned by the National Institute of Diabetes and Digestive and Kidney Diseases.
This particular subset deals with specifically with women over the age of 21 of the Pima Indian heritage.
The dataset contains the following columns:

* `Pregnancies` - the number of pregnancies the woman has gone through.
* `Glucose` - Glucose level.
* `BloodPressure` - Diastolic blood pressure in mm Hg.
* `SkinThickness` - Triceps skin fold thickness in mm
* `Insulin` - 2-Hour serum insulin test result in mu U/ml 
* `BMI` - Body mass index
* `DiabetesPedigreeFunction` - Diabetes pedigree function 
* `Age` - The age
* `Outcome` - Outcome where 1 indicates a positive test result.


As usual, we will be loading the dataset through pandas.

In [ ]:
import pandas as pd
data = pd.read_csv('diabetes.csv')
data.head()

Let's take a quick look at the dataset information to undestand the column compostion.


In [ ]:
data.info()

Every single column in the dataset is numeric.
Let us take a look at the distribution of the data in each column.


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.color_palette("rocket", as_cmap=True)
for col in data.columns:
    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(15,7))
    ax1.boxplot(data[col])
    sns.kdeplot(data[col], ax=ax2)
    plt.title(col)
plt.show()

We can see that certain values here were imputed as some of the values can never be equal to 0 in the real world.
For example, neither BMI or blood preassure can be equal to 0, yet we do see some values in the columns that are equal to 0.
We could impute these values using a mean, median, or perhaps even KNN-imputer in order to try and improve the performance of the model.
Yet surprisingly, when I tried to do so, the models performed worse when I tried to impute these value.
So, they will stay as they are.
Next, we will look at the distributions of the data per outcome.

In [ ]:
for col in data.columns:
    if col == "Outcome":
        continue
    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(15,7))
    positives = data["Outcome"] == 1
    negatives = data["Outcome"] == 0
    ax1.boxplot(data[col].loc[positives], positions=[1])
    ax1.boxplot(data[col].loc[negatives], positions=[2])
    ax1.set_xticklabels(["Positive", "Negative"])
    ax1.set_title(f"{col} Boxplot")
    sns.kdeplot(data[col].loc[positives], ax=ax2, label="Positive")
    sns.kdeplot(data[col].loc[negatives], ax=ax2, label="Negative")
    plt.legend(loc="best")
    plt.title(f"{col} KDE")
plt.show()

Inrestingly, the distributions for both the blood pressure and skin thickness appear to be very similar to each other regardless of the outcome.
We could experiment and try to remove these variables from the model down the line and see if we are able to achieve better performance.
For now, let's check if we can spot anything interesting happening when we project the data into 2 dimensions.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

y = data["Outcome"]
X = data.drop("Outcome", axis=1)

pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=2)),
])

pca_data = pd.DataFrame(
    pipeline.fit_transform(X),
    columns=["PC1", "PC2"],
    index=data.index,
)

plt.scatter(pca_data["PC1"], pca_data["PC2"], c=y, cmap="viridis")
plt.title("Projection of the Diabetes Data")

Unfortunatley, the difference between the two distributions is a bit difficult to spot in these two dimension.

## Training Naive Bayes
We will now proceed to train and test the naive bayes model.
As usual, we should split the data into train and test sets.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Alright, now let us train the model, but first - some theory.

As mentioned above, naive bayes is a probabilistic classifier, so it should be able to output the probabilities of each outcome along with its prediction.

It does so by computing $P(Y | X)$ where $Y$ represents the classified variable and X represents the feature variables.
According to Bayes theorem (that's where the method get's its name):
$$
P(Y | X) = \frac{P(X | Y) P(Y)}{P(X)}
$$
Although this equation looks simple, it becomes very difficult to compute as $X$ grows larger.
This is because we have to calculate $P(X | Y)$.
For example, assume X consists of two features $X_1$ and $X_2$.
Then, we have to calculate 
$$
P(Y | X_1, X_2) = \frac{P(X_1, X_2 | Y) P(Y)}{P(X_1, X_2)}
$$
So, now we need data where all the three variables ($X_1$, $X_2$, $Y$) interact.
However, naive bayes makes a very crucial assumption.
It assumes that the variables are independent.
Thus, our equation is simplified to 
$$
P(Y | X_1, X_2) = P(X_1 | Y) P(X_2 | Y)
$$
Now, we no longer need the data on how $X_1$ and $X_2$ interact with each other and only need to know how they interact with $Y$.
This means that we need significantly less data and the computation becomes much cheaper.

You may be concerned that the assumption of independnce is flawed since variables in most data are going to have some degree of dependence on each other.
That is a good concern to have.
However, naive bayes generally tends to perform well even when that assumption is violated.

So, when given observations $X_1, X_2, ..., X_n$, naive baise calculates the probabilities for all the possible outcomes $Y$ and then selects the outcome with the highest probability. 
This is how we both get a probability estimate and a prediction. 

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, f1_score, accuracy_score
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(X_train, y_train)
prediction = nb.predict(X_test)
cmd = ConfusionMatrixDisplay.from_estimator(nb, X_test, y_test)
print("Accuracy: ", accuracy_score(prediction, y_test))
print("F1 score: ", f1_score(prediction, y_test))

In [ ]:
x1 = X_test.iloc[0]
x1

In [ ]:
x1 = x1.to_numpy().reshape(1, -1)
prediction = nb.predict(x1)
probs = nb.predict_proba(x1)[0]
print(
    f"Naive bayes predicted that the outcome would be {prediction[0]} with probability of {max(probs)}"

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)
cmd = ConfusionMatrixDisplay.from_estimator(lr, X_test, y_test)
prediction = lr.predict(X_test)
print("Accuracy: ", accuracy_score(prediction, y_test))
print("F1 score: ", f1_score(prediction, y_test))